In [15]:
import pandas as pd
import numpy as np
import warnings

# On ignore les messages rouges non critiques
warnings.filterwarnings('ignore')

# --- CONFIGURATION ---
TRAIN_PATH = "/Users/alexandre/Desktop/X/Python for Data Science/Projet Final Churn/train.parquet"
OUTPUT_PATH = "train_features_optimized.parquet"

# T0 : La date de coupure. On regarde l'historique AVANT cette date.
T0 = pd.Timestamp("2018-11-10") 
HORIZON_DAYS = 10 # On cherche à prédire le churn dans les 10 jours qui suivent

print("✅ Configuration chargée.")


✅ Configuration chargée.


In [16]:
print("⏳ Chargement du fichier train...")
# Chargement
df = pd.read_parquet(TRAIN_PATH)

# Conversion des dates (millisecondes -> datetime)
df["ts"] = pd.to_datetime(df["ts"], unit="ms")
df["date"] = df["ts"].dt.date

print(f"📊 Dimensions du dataset : {df.shape}")
df.head(3)


⏳ Chargement du fichier train...
📊 Dimensions du dataset : (17499636, 20)


,status,gender,firstName,level,lastName,userId,ts,auth,page,sessionId,location,itemInSession,userAgent,method,length,song,artist,time,registration,date
0,200,M,Shlok,paid,Johnson,1749042,2018-10-01 00:00:01,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",278,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,524.32934,Ich mache einen Spiegel - Dream Part 4,Popol Vuh,2018-10-01 00:00:01,2018-08-08 13:22:21,2018-10-01
992,200,M,Shlok,paid,Johnson,1749042,2018-10-01 00:08:45,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",279,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,178.02404,Monster (Album Version),Skillet,2018-10-01 00:08:45,2018-08-08 13:22:21,2018-10-01
1360,200,M,Shlok,paid,Johnson,1749042,2018-10-01 00:11:43,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",280,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,232.61995,Seven Nation Army,The White Stripes,2018-10-01 00:11:43,2018-08-08 13:22:21,2018-10-01


In [17]:
print("✂️ Découpage temporel (Observation vs Futur)...")

# 1. On sépare l'historique (ce qu'on connaît)
obs = df[df["ts"] <= T0].copy()

# 2. On isole le futur (ce qu'on veut prédire)
future = df[(df["ts"] > T0) & (df["ts"] <= T0 + pd.Timedelta(days=HORIZON_DAYS))].copy()

# 3. NETTOYAGE : On retire les utilisateurs qui ont DÉJÀ annulé avant le 10 nov.
# On ne peut pas prédire le départ de quelqu'un qui est déjà parti !
past_churners = obs[obs["page"] == "Cancellation Confirmation"]["userId"].unique()
obs_clean = obs[~obs["userId"].isin(past_churners)].copy()

# Liste des utilisateurs restants à analyser
users_population = obs_clean["userId"].unique()

print(f"👥 Nombre d'utilisateurs actifs au T0 : {len(users_population)}")

✂️ Découpage temporel (Observation vs Futur)...
👥 Nombre d'utilisateurs actifs au T0 : 15276


In [18]:
# Identification des churners dans le futur
churners_future = future[future["page"] == "Cancellation Confirmation"]["userId"].unique()

# Création du DataFrame final avec la colonne 'target'
target_df = pd.DataFrame({"userId": users_population})

# Si l'user est dans la liste des churners futurs, target = 1, sinon 0
target_df["target"] = target_df["userId"].isin(churners_future).astype(int)

print("🎯 Distribution de la cible (Combien de churners ?) :")
print(target_df["target"].value_counts())


🎯 Distribution de la cible (Combien de churners ?) :
target
0    14613
1      663
Name: count, dtype: int64


In [19]:
print("🏗️ Calcul des features globales...")

global_feats = obs_clean.groupby("userId").agg({
    "ts": "max",                      # Date de dernière action
    "date": "nunique",                # Nombre de jours actifs totaux
    "sessionId": "nunique",           # Nombre de sessions totales
    "length": "sum",                  # Temps total d'écoute
    "registration": "min"             # Date d'inscription
}).reset_index()

global_feats.columns = ["userId", "last_ts", "n_active_days", "n_sessions", "total_listening_time", "registration_ts"]

# Conversion date inscription
global_feats["registration_ts"] = pd.to_datetime(global_feats["registration_ts"], unit="ms")

# Feature 1 : Récence (Jours écoulés depuis la dernière action avant T0)
global_feats["recency_days"] = (T0 - global_feats["last_ts"]).dt.days

# Feature 2 : Ancienneté du compte en jours
global_feats["account_age_days"] = (T0 - global_feats["registration_ts"]).dt.days

# Feature 3 : Temps d'écoute moyen par jour d'ancienneté
global_feats["avg_daily_listen"] = global_feats["total_listening_time"] / (global_feats["account_age_days"] + 1)

display(global_feats.head())


🏗️ Calcul des features globales...


,userId,last_ts,n_active_days,n_sessions,total_listening_time,registration_ts,recency_days,account_age_days,avg_daily_listen
0,1000035,2018-11-08 03:51:48,17,16,210659.31890,2018-09-12 19:28:22,1,58,3570.496931
1,1000103,2018-11-08 18:28:40,3,3,13554.73009,2018-09-22 07:27:25,1,48,276.627145
2,1000164,2018-11-09 12:03:36,12,11,133275.57835,2018-08-12 09:32:01,0,89,1480.839759
3,1000168,2018-11-08 17:27:42,7,6,121243.91254,2018-08-08 16:06:13,1,93,1289.828857
4,1000182,2018-11-09 23:50:22,7,4,87856.68704,2018-07-03 12:06:05,0,129,675.820670


In [20]:
print("👍 Calcul des indicateurs de comportement (Likes, Erreurs)...")

# Pivot table : crée une colonne pour chaque type de page
page_counts = pd.pivot_table(
    obs_clean, 
    index="userId", 
    columns="page", 
    values="ts", 
    aggfunc="count", 
    fill_value=0
).reset_index()

# On sélectionne seulement les pages utiles
useful_pages = ["Thumbs Up", "Thumbs Down", "Roll Advert", "Error", "Upgrade", "Downgrade", "Add to Playlist"]
cols_to_keep = ["userId"] + [col for col in useful_pages if col in page_counts.columns]
behavior_df = page_counts[cols_to_keep].copy()

# Ratio de Satisfaction : (Likes) / (Dislikes + 1)
if "Thumbs Up" in behavior_df and "Thumbs Down" in behavior_df:
    behavior_df["satisfaction_ratio"] = behavior_df["Thumbs Up"] / (behavior_df["Thumbs Down"] + 1)

display(behavior_df.head())



👍 Calcul des indicateurs de comportement (Likes, Erreurs)...


page,userId,Thumbs Up,Thumbs Down,Roll Advert,Error,Upgrade,Downgrade,Add to Playlist,satisfaction_ratio
0,1000035,75,11,5,0,5,2,20,6.250000
1,1000103,2,1,3,0,1,1,1,1.000000
2,1000164,18,4,20,1,1,6,14,3.600000
3,1000168,55,3,2,0,1,2,15,13.750000
4,1000182,16,6,0,0,1,5,12,2.285714


In [21]:
print("📈 Calcul des tendances (Activité récente vs Habitude)...")

# 1. On prend seulement les logs des 14 derniers jours avant T0
T_recent = T0 - pd.Timedelta(days=14)
obs_recent = obs_clean[obs_clean["ts"] >= T_recent]

# 2. On calcule le temps d'écoute sur cette période récente
recent_stats = obs_recent.groupby("userId").agg({
    "length": "sum"     
}).reset_index().rename(columns={"length": "listen_time_recent"})

# 3. On merge avec les stats globales pour comparer
trends = global_feats[["userId", "avg_daily_listen"]].merge(recent_stats, on="userId", how="left").fillna(0)

# 4. Moyenne quotidienne RÉCENTE
trends["avg_daily_listen_recent"] = trends["listen_time_recent"] / 14

# 5. RATIO (TREND) : Récent / Habitude
# Si < 1 : L'utilisateur ralentit -> Risque de Churn
trends["trend_listening"] = trends["avg_daily_listen_recent"] / (trends["avg_daily_listen"] + 0.01)

display(trends[["userId", "trend_listening"]].head())


📈 Calcul des tendances (Activité récente vs Habitude)...


,userId,trend_listening
0,1000035,1.638383
1,1000103,0.254093
2,1000164,2.422537
3,1000168,1.943355
4,1000182,6.396788


In [23]:
# === A AJOUTER DANS NOTEBOOK 01 (Avant la fusion finale) ===
print("💻 Extraction des features techniques (OS & Device)...")

# On prend le dernier userAgent connu pour chaque utilisateur
last_agent = obs_clean.sort_values("ts").groupby("userId")["userAgent"].last().reset_index()

# Création manuelle des flags (plus sûr que get_dummies pour la compatibilité Train/Test)
# 1. Système d'exploitation
last_agent["is_mac"] = last_agent["userAgent"].str.contains("Macintosh", case=False, na=False).astype(int)
last_agent["is_windows"] = last_agent["userAgent"].str.contains("Windows", case=False, na=False).astype(int)
last_agent["is_linux"] = last_agent["userAgent"].str.contains("Linux", case=False, na=False).astype(int)
last_agent["is_mobile"] = last_agent["userAgent"].str.contains("iPhone|iPad|Android|Mobile", case=False, na=False).astype(int)

# 2. Navigateur (les utilisateurs Chrome/Firefox ont souvent des profils différents des utilisateurs IE/Safari)
last_agent["is_firefox"] = last_agent["userAgent"].str.contains("Firefox", case=False, na=False).astype(int)
last_agent["is_chrome"] = last_agent["userAgent"].str.contains("Chrome", case=False, na=False).astype(int)

# On garde uniquement les nouvelles colonnes
tech_features = last_agent[["userId", "is_mac", "is_windows", "is_linux", "is_mobile", "is_firefox", "is_chrome"]]

print(f"✅ Features techniques prêtes. Shape : {tech_features.shape}")

💻 Extraction des features techniques (OS & Device)...
✅ Features techniques prêtes. Shape : (15276, 7)


In [24]:
print("🧩 Fusion finale des features...")

# On part de la target et on ajoute tout
final_df = target_df.merge(global_feats, on="userId", how="left")
final_df = final_df.merge(behavior_df, on="userId", how="left")
final_df = final_df.merge(trends[["userId", "trend_listening"]], on="userId", how="left")
final_df = final_df.merge(tech_features, on="userId", how="left").fillna(0)
# Remplacer les vides par 0
final_df = final_df.fillna(0)

# Nettoyage des colonnes dates inutiles pour le modèle
cols_to_drop = ["last_ts", "registration_ts"]
final_df = final_df.drop(columns=[c for c in cols_to_drop if c in final_df.columns])

print(f"✅ Terminé ! Shape finale : {final_df.shape}")
final_df.to_parquet(OUTPUT_PATH, index=False)
print(f"💾 Fichier sauvegardé : {OUTPUT_PATH}")


🧩 Fusion finale des features...
✅ Terminé ! Shape finale : (15276, 23)
💾 Fichier sauvegardé : train_features_optimized.parquet
